# Práctica 4. Redes Convolucionales

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

from keras.datasets import mnist

num_classes = 10
input_shape = (28, 28, 1)

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



## Red Neuronal Convolucional 

In [2]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

In [3]:
batch_size = 128
epochs = 10

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)



Epoch 1/10
422/422 [==============================] - 11s 25ms/step - loss: 0.3651 - accuracy: 0.8896 - val_loss: 0.0871 - val_accuracy: 0.9757
Epoch 2/10
422/422 [==============================] - 10s 24ms/step - loss: 0.1164 - accuracy: 0.9639 - val_loss: 0.0589 - val_accuracy: 0.9840
Epoch 3/10
422/422 [==============================] - 10s 23ms/step - loss: 0.0857 - accuracy: 0.9735 - val_loss: 0.0473 - val_accuracy: 0.9868
Epoch 4/10
422/422 [==============================] - 11s 27ms/step - loss: 0.0705 - accuracy: 0.9784 - val_loss: 0.0423 - val_accuracy: 0.9877
Epoch 5/10
422/422 [==============================] - 13s 30ms/step - loss: 0.0628 - accuracy: 0.9803 - val_loss: 0.0394 - val_accuracy: 0.9878
Epoch 6/10
422/422 [==============================] - 14s 32ms/step - loss: 0.0564 - accuracy: 0.9824 - val_loss: 0.0411 - val_accuracy: 0.9878
Epoch 7/10
422/422 [==============================] - 13s 32ms/step - loss: 0.0514 - accuracy: 0.9841 - val_loss: 0.0348 - val_accuracy:

In [4]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])
print("Test error rate:", (1 - score[1])*100)




Test loss: 0.029796628281474113
Test accuracy: 0.9891999959945679
Test error rate: 1.080000400543213


In [5]:
import numpy as np

ohe_predicted = model.predict(x_test)
predicted_labels = ohe_predicted.argmax(axis=1)

final_result = ""

for v in predicted_labels:
    final_result += str(v)

with open('final.txt', 'w', encoding='utf-8') as f:
    f.write(final_result)

313/313 [==============================] - 1s 4ms/step


## Ajuste de hiper-parámetros con Keras-Tuner

In [6]:
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner

def call_existing_code(units, activation, dropout, lr):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(layers.Dense(units=units, activation=activation))
    if dropout:
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


def build_model(hp):
    units = hp.Int("units", min_value=32, max_value=512, step=32)
    activation = hp.Choice("activation", ["relu", "tanh"])
    dropout = hp.Boolean("dropout")
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    # call existing model-building code with the hyperparameter values.
    model = call_existing_code(
        units=units, activation=activation, dropout=dropout, lr=lr
    )
    return model


build_model(keras_tuner.HyperParameters())

In [7]:
(x, y), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))

Trial 3 Complete [00h 00m 11s]
val_accuracy: 0.9591000080108643

Best val_accuracy So Far: 0.9591000080108643
Total elapsed time: 00h 00m 56s


In [8]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(None, 28, 28))
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 96)                75360     
                                                                 
 dropout (Dropout)           (None, 96)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 76330 (298.16 KB)
Trainable params: 76330 (298.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
tuner.results_summary()

Results summary
Results in my_dir\helloworld
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
units: 96
activation: tanh
dropout: True
lr: 0.004342864594750216
Score: 0.9591000080108643

Trial 0 summary
Hyperparameters:
units: 416
activation: tanh
dropout: False
lr: 0.009471858758984132
Score: 0.9406999945640564

Trial 1 summary
Hyperparameters:
units: 160
activation: tanh
dropout: True
lr: 0.0001508693083119637
Score: 0.9245000183582306


### Re-entranando el modelo

In [ ]:
# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_model(best_hps[0])
# Fit with the entire dataset.
x_all = np.concatenate((x_train, x_val))
y_all = np.concatenate((y_train, y_val))
model.fit(x=x_all, y=y_all, epochs=1)

score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])
print("Test error rate:", (1 - score[1])*100)



In [ ]:
tuner.oracle.trials['0'].metrics.metrics['val_loss'].get_statistics()


## Ajuste de parámetros con apoyo de Scikit-Learn

In [10]:
import keras
from tensorflow.keras import layers
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV, RepeatedKFold
import numpy as np

input_shape = (28, 28, 1)
num_classes = 10

(x, y), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



 
# Function to create model, required for KerasClassifier
def create_model(kernel_size=(2,2), pool_size=(2,2), rate=0.5):
    # create model
    model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=kernel_size, activation="relu"),
        layers.MaxPooling2D(pool_size=pool_size),
        layers.Conv2D(64, kernel_size=kernel_size, activation="relu"),
        layers.MaxPooling2D(pool_size=pool_size),
        layers.Flatten(),
        layers.Dropout(rate=rate),
        layers.Dense(num_classes, activation="softmax"),
    ])
    # Compile model
    
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model
 
# fix random seed for reproducibility
seed = 123
np.random.seed(seed)

# create model
model = KerasClassifier(model=create_model, verbose=0)
print(model.get_params().keys())

kernel_sizes = [(2,2), (3,3)]
pool_sizes = [(2,2)]
rates = [0.5, 0.75]

epochs = [10]
batches = [128]

param_grid = dict(model__kernel_size=kernel_sizes, 
                  epochs=epochs, 
                  batch_size=batches, 
                  model__pool_size=pool_sizes,
                  model__rate=rates)

cv = RepeatedKFold(n_splits=3, n_repeats=2, random_state=123)

search = GridSearchCV(estimator=model, 
                      param_grid=param_grid, 
                      scoring="accuracy", 
                      cv=cv, 
                      n_jobs=15)

grid_result = search.fit(X=x_train, y=y_train)











dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'class_weight'])


In [8]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.990810 using {'batch_size': 128, 'epochs': 50, 'model__kernel_size': (3, 3), 'model__pool_size': (2, 2), 'model__rate': 0.5}
0.987420 (0.000781) with: {'batch_size': 128, 'epochs': 50, 'model__kernel_size': (2, 2), 'model__pool_size': (2, 2), 'model__rate': 0.5}
0.987060 (0.001312) with: {'batch_size': 128, 'epochs': 50, 'model__kernel_size': (2, 2), 'model__pool_size': (2, 2), 'model__rate': 0.75}
0.990810 (0.001044) with: {'batch_size': 128, 'epochs': 50, 'model__kernel_size': (3, 3), 'model__pool_size': (2, 2), 'model__rate': 0.5}
0.990630 (0.000688) with: {'batch_size': 128, 'epochs': 50, 'model__kernel_size': (3, 3), 'model__pool_size': (2, 2), 'model__rate': 0.75}


In [ ]:
y_train